In [89]:
import pandas as pd
import numpy as np
from random import randrange
import nltk
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import requests

,author,quote
0,Michael Jordan,I've missed more than 9000 shots in my career....
1,Henry David Thoreau,Many men go fishing all of their lives without...
2,Vince Lombardi,Winners never quit and quitters never win.
3,Babe Ruth,The way a team plays as a whole determines its...
4,Bob Feller,Every day is a new opportunity. You can build ...
...,...,...
105,Winston Churchill,History is written by the victors.
106,Winston Churchill,"The farther backward you can look, the farther..."
107,Winston Churchill,I am an optimist. It does not seem too much us...
108,Winston Churchill,"It is no use saying, 'We are doing our best.' ..."


In [75]:
# Defining the url of the site
base_site = "http://api.quotable.io/quotes?limit=110&maxLength=262"

# Making a get request
response = requests.get(base_site)
response.status_code


200

In [76]:
response.json()

{'count': 110,
 'totalCount': 2031,
 'page': 1,
 'totalPages': 19,
 'lastItemIndex': 110,
 'results': [{'_id': 'JIk3cbQ8s',
   'author': 'Michael Jordan',
   'content': "I've missed more than 9000 shots in my career. I've lost almost 300 games. 26 times, I've been trusted to take the game winning shot and missed. I've failed over and over and over again in my life. And that is why I succeed.",
   'tags': ['sports', 'competition', 'famous-quotes'],
   'authorSlug': 'michael-jordan',
   'length': 224,
   'dateAdded': '2022-07-06',
   'dateModified': '2022-07-08'},
  {'_id': 'JRmne2RXYn',
   'author': 'Henry David Thoreau',
   'content': 'Many men go fishing all of their lives without knowing that it is not fish they are after.',
   'tags': ['sports', 'competition'],
   'authorSlug': 'henry-david-thoreau',
   'length': 90,
   'dateAdded': '2022-07-06',
   'dateModified': '2022-07-08'},
  {'_id': '0Ny0rPtnM3',
   'author': 'Vince Lombardi',
   'content': 'Winners never quit and quitters ne

In [77]:
r = response.json()

In [78]:
our_data = pd.DataFrame(r['results'], columns=['author', 'content'])

In [79]:
our_data[['quote', 'author']] = our_data[['content', 'author']]

In [80]:
our_data = our_data.drop('content', axis=1)

In [81]:
our_data

,author,quote
0,Michael Jordan,I've missed more than 9000 shots in my career....
1,Henry David Thoreau,Many men go fishing all of their lives without...
2,Vince Lombardi,Winners never quit and quitters never win.
3,Babe Ruth,The way a team plays as a whole determines its...
4,Bob Feller,Every day is a new opportunity. You can build ...
...,...,...
105,Winston Churchill,History is written by the victors.
106,Winston Churchill,"The farther backward you can look, the farther..."
107,Winston Churchill,I am an optimist. It does not seem too much us...
108,Winston Churchill,"It is no use saying, 'We are doing our best.' ..."


In [93]:



# declare global variable
quotes = None

def prepare_sentiment_quote_stash(quote_stash_path):
    global quotes

    response = requests.get("https://api.quotable.io/quotes?limit=110&maxLength=262")
    r = response.json()
    our_data = pd.DataFrame(r['results'], columns=['author', 'content'])
    our_data[['quote', 'author']] = our_data[['content', 'author']]
    our_data = our_data.drop('content', axis=1)

    # load the quote stash
    quotes = our_data

    sid = SentimentIntensityAnalyzer()

    all_compounds = []
    for sentence in quotes['quote']:
        ss = sid.polarity_scores(sentence)
        for k in sorted(ss):
            if k == 'compound':
                all_compounds.append(ss[k])


    # add sentiment to the data
    quotes['sentiment_score'] = all_compounds

    # create ladder index
    quotes = quotes.sort_values('sentiment_score')
    quotes['index'] = [ix for ix in range(0, len(quotes))]

    return quotes


# load the quote stash
quotes = prepare_sentiment_quote_stash('quotes.csv')

# sest max quote index value
max_index_value = np.max(quotes['index'].values)

def gimme_a_quote(direction = None, current_index = None):
    rand_index = randrange(max_index_value)
    darker = None
    brighter = None


    # New session visit
    if current_index is None:
        brighter = rand_index

    if direction == 'brighter':
        brighter = current_index
    else:
        darker = current_index

    if darker is not None:
        current_index = rand_index

        try:
            current_index = int(darker)
        except ValueError:
            # somebody is gaming the system
            current_index = rand_index


        if current_index > 0:
            # try for a lesser value than current one
            rand_index = randrange(0, current_index)
        else:
            # already at lowest point so assign a new random of full set
            rand_index = rand_index


    elif brighter is not None:

        try:
            current_index = int(brighter)
        except ValueError:
            # somebody is gaming the system
            current_index = rand_index

        # try for a higher value than current one
        if current_index < max_index_value -1:
            rand_index = randrange(current_index, max_index_value)
        else:
            # already at highest point so assign a new random of full set
            rand_index = rand_index
    else:
        # grab a random value
        rand_index = rand_index

    return (rand_index)


quote_number = gimme_a_quote('brighter', 20)
quotes[quotes['index'] == gimme_a_quote('brighter', quote_number) ]

,author,quote,sentiment_score,index
20,Casey Stengel,Finding good players is easy. Getting them to ...,0.802,103
